In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.ticker as ticker   


# setup spark
conf = SparkConf().setAppName('WikiGraph')
sc = SparkContext(conf=conf)
#from P5_bfs import *

import pandas as pd

In [5]:
# paths to the big data files
datapath = '../../../../../courses/CS205_Computing_Foundations/data/'
titlespath = datapath + 'titles-sorted.txt'
linkspath = datapath + 'links-simple-sorted.txt'

In [6]:
rddTitles = sc.textFile(titlespath)
rddGraph = sc.textFile(linkspath)

# prepare Graph into adjacency list structure
rddGraph = rddGraph.map(lambda x: x.replace(':', '').split(' ')).map(lambda x: (int(x[0]), [int(y) for y in x[1:]]))

In [4]:
rddTitles.take(5)


[u'!', u'!!', u'!!!', u'!!!!', u'!!!Fuck_You!!!']

In [5]:
rddTitles = rddTitles.zipWithIndex().map(lambda x: (x[0], x[1] + 1)).cache()

In [6]:
rddTitles.take(5)

[(u'!', 1), (u'!!', 2), (u'!!!', 3), (u'!!!!', 4), (u'!!!Fuck_You!!!', 5)]

In [7]:
# lookup a title (its index)
keyString = 'Kevin_Bacon'
v0 = rddTitles.filter(lambda x: x[0] == keyString).collect()[0][1]

keyString = 'Harvard_University'
vT = rddTitles.filter(lambda x: x[0] == keyString).collect()[0][1] 

In [8]:
v0

2729536

In [9]:
vT

2152782

In [10]:
# compute shortest path!
from P5_sssp import *

In [11]:
num_visited_nodes, rddPaths = sparkSSSP(sc, rddGraph, v0, vT, 20)

In [12]:
resultPaths = rddPaths.collect()

In [13]:
resultPaths

[[2729536, 3229511, 2152782],
 [2729536, 4625677, 2152782],
 [2729536, 2578703, 2152782],
 [2729536, 5114592, 2152782],
 [2729536, 1124925, 2152782]]

In [14]:
rddLookup = sc.parallelize(resultPaths)

In [15]:
rddLookup.take(5)

[[2729536, 3229511, 2152782],
 [2729536, 4625677, 2152782],
 [2729536, 2578703, 2152782],
 [2729536, 5114592, 2152782],
 [2729536, 1124925, 2152782]]

In [23]:
#rddLookup.zipWithIndex().flatMap(lambda x: [(y, x[1]+1) for y in x[0]]).join(rddTitles.map(lambda x: (x[1], x[0]))).map(lambda x: (x[1][0], x[1][1])).groupByKey().map(lambda x: list(x[1])).collect()

[[u'Kevin_Bacon', u'Harvard_University', u'Marisa_Silver'],
 [u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'],
 [u'Kevin_Bacon', u'Harvard_University', u'John_Lithgow'],
 [u'Time_(magazine)', u'Kevin_Bacon', u'Harvard_University'],
 [u'Kevin_Bacon', u'College_Bowl', u'Harvard_University']]

In [41]:
rddLookup.zipWithIndex().flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)])rddLookup.zipWithIndex().flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)]).collect()

[(2729536, (1, 1)),
 (3229511, (1, 2)),
 (2152782, (1, 3)),
 (2729536, (2, 1)),
 (4625677, (2, 2)),
 (2152782, (2, 3)),
 (2729536, (3, 1)),
 (2578703, (3, 2)),
 (2152782, (3, 3)),
 (2729536, (4, 1)),
 (5114592, (4, 2)),
 (2152782, (4, 3)),
 (2729536, (5, 1)),
 (1124925, (5, 2)),
 (2152782, (5, 3))]

In [45]:
# for better performance on join, 
rddLookup.cache()
nodesOfInterest = rddLookup.flatMap(lambda x: x).distinct().collect()
selectedTitles = rddTitles.filter(lambda x: x[1] in nodesOfInterest).map(lambda x: (x[1], x[0]))

In [56]:
rddLookup.zipWithIndex() \
         .flatMap(lambda x: [(y, (x[1]+1, i)) for i, y in enumerate(x[0], 1)]) \
         .join(selectedTitles) \
         .map(lambda x: (x[1][0][0], (x[1][0][1], x[1][1]))) \
         .groupByKey().map(lambda x: zip(*sorted(list(x[1])))[1]).collect()
         

[(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [58]:
test = [(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [59]:
test

[(u'Kevin_Bacon', u'Marisa_Silver', u'Harvard_University'),
 (u'Kevin_Bacon', u'Six_degrees_of_separation', u'Harvard_University'),
 (u'Kevin_Bacon', u'John_Lithgow', u'Harvard_University'),
 (u'Kevin_Bacon', u'Time_(magazine)', u'Harvard_University'),
 (u'Kevin_Bacon', u'College_Bowl', u'Harvard_University')]

In [69]:
startTitle  = 'Kevin_Bacon'
targetTitle = 'Harvard_University'

with open('output.txt', 'wb') as f:
	f.write('Shortest paths from %s to %s:\n\n' % (startTitle, targetTitle))
	for path in test:
            path = list(path) 
            f.write(path[0])
            for node in path[1:]:
                f.write(' -> ' + node)
            f.write('\n')

# Connected Components for Wikigraph

In [2]:
# perform PEGASUS algorithm after ...
def connectedComponents(rddIn):
    
    # transform to structure (init)
    rdd = rddIn.map(lambda x: (x[0], (x[1], x[0])))
    maxNodeID = rdd.map(lambda x: x[0]).max()
    minNodeID = rdd.map(lambda x: x[0]).min()
    
    # helper function to check whether the partitioning in connected components changed
    def checkForPartitionChange(newSizes, oldSizes):
        partitionChanged = False

        # quick check if length of the dict is different to previous step
        # the partitions changed!
        if len(oldSizes.items()) != len(newSizes.items()):
            partitionChanged = True
        else:
            # the size did not change, but what about some internal shifting?
            # ==> check for each connected component if size changed!
            for ID in range(minNodeID, maxNodeID+1):
                # check if for old / new the current ID exists and is equal
                # ==> if not, a change happened!
                try:
                    sizeOld = oldSizes[ID]
                    sizeNew = newSizes[ID]

                    if sizeOld != sizeNew:
                        partitionChanged = True
                        break
                except KeyError:
                    partitionChanged = True
                    break

        return partitionChanged
    
    
    # this is the algorithm (Pegasus after ...)
    done = False
    
    counter = 0;
    while not done:
        # compute sizes of connected components
        rddComponents = rdd.map(lambda x: x[1][1])
        oldSizes = rddComponents.countByValue()


        # mapper / expander
        rdd = rdd.flatMap(lambda x: [x] + [(y, ([], x[1][1])) for y in x[1][0]])

        # reducer (as there is only !one! element with the whole adj. list, we can speed it up here!)
        rdd = rdd.reduceByKey(lambda a,b: (a[0] + b[0], min(a[1], b[1])))

        # determine if number of partitions changed! (0 is a dummy value to construct an pair rdd)
        rddComponents = rdd.map(lambda x: x[1][1])
        newSizes = rddComponents.countByValue()

        # if partition changed, one more round!
        done = not checkForPartitionChange(newSizes, oldSizes)

        oldSizes = newSizes
        
        counter += 1

    # reconstruct connected components
    componentList = rdd.map(lambda x: (x[1][1], x[0])).sortByKey().groupByKey().map(lambda x: sorted(list(x[1]))).collect()

    return counter, componentList

In [8]:
# all in one function
def directedGraph2symmetric(rdd):
    
    return rdd.flatMap(lambda x: [x] + [(y, [x[0]]) for y in x[1]]) \
              .sortByKey() \
              .reduceByKey(lambda a,b: a+b) \
              .map(lambda x: (x[0], list(set(x[1]))))

In [4]:
# all in one function\
def directedGraph2linked(rdd, includeSingleComponents=True):
    rddSingleNodes = None
    if includeSingleComponents:
        rddSingleNodes = rdd.map(lambda x: (x[0], []))
        
    rdd = rdd.flatMap(lambda x: [(x[0], y) for y in x[1]])
    rdd = rdd.map(lambda x: ((min(x[0], x[1]), max(x[0], x[1])), 1))
    rdd = rdd.reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] > 1).map(lambda x: x[0])
    rdd = rdd.flatMap(lambda x: [(x[0], [x[1]]), (x[1], [x[0]])])
    
    if includeSingleComponents:
        # if desired, join with single nodes
        rdd = rdd.union(rddSingleNodes)

    rdd = rdd.reduceByKey(lambda a, b: a+b)
    
    return rdd

## experiment 1 - symmetric graph

In [9]:
# paths to the big data files
datapath = '../../../../../courses/CS205_Computing_Foundations/data/'
titlespath = datapath + 'titles-sorted.txt'
linkspath = datapath + 'links-simple-sorted.txt'

rddTitles = sc.textFile(titlespath)
rddGraph = sc.textFile(linkspath)

# prepare Graph into adjacency list structure
rddGraph = rddGraph.map(lambda x: x.replace(':', '').split(' ')).map(lambda x: (int(x[0]), [int(y) for y in x[1:]])).cache()
# prepare Title dictionary
rddTitles = rddTitles.zipWithIndex().map(lambda x: (x[0], x[1] + 1)).cache()

In [10]:
# convert to symmetric Graph
rddUD = directedGraph2symmetric(rddGraph)
mappedGraph = rddUD.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 6.0 failed 1 times, most recent failure: Lost task 2.0 in stage 6.0 (TID 82, localhost): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRDD$$anon$1.read(PythonRDD.scala:113)
	at org.apache.spark.api.python.PythonRDD$$anon$1.next(PythonRDD.scala:101)
	at org.apache.spark.api.python.PythonRDD$$anon$1.next(PythonRDD.scala:97)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at scala.collection.Iterator$GroupedIterator.takeDestructively(Iterator.scala:914)
	at scala.collection.Iterator$GroupedIterator.go(Iterator.scala:929)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:969)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:972)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:327)
	at or

In [ ]:
counter, cList = connectedComponents(rddUD)

# experiment - links 

In [ ]:
# paths to the big data files
datapath = '../../../../../courses/CS205_Computing_Foundations/data/'
titlespath = datapath + 'titles-sorted.txt'
linkspath = datapath + 'links-simple-sorted.txt'

rddTitles = sc.textFile(titlespath)
rddGraph = sc.textFile(linkspath)

# prepare Graph into adjacency list structure
rddGraph = rddGraph.map(lambda x: x.replace(':', '').split(' ')).map(lambda x: (int(x[0]), [int(y) for y in x[1:]]))
# prepare Title dictionary
rddTitles = rddTitles.zipWithIndex().map(lambda x: (x[0], x[1] + 1)).cache()

In [ ]:
# convert to symmetric Graph
rddUD2 = directedGraph2linked(rddGraph, True)

counter2, cList2 = connectedComponents(rddUD2)